In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 6)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.292161643505096, Final Batch Loss: 0.9976381659507751
Epoch 2, Loss: 4.375752091407776, Final Batch Loss: 1.0886656045913696
Epoch 3, Loss: 4.346004009246826, Final Batch Loss: 1.0643161535263062
Epoch 4, Loss: 4.315779328346252, Final Batch Loss: 1.0391725301742554
Epoch 5, Loss: 4.278451085090637, Final Batch Loss: 1.0078141689300537
Epoch 6, Loss: 4.545019745826721, Final Batch Loss: 1.2809569835662842
Epoch 7, Loss: 4.48827052116394, Final Batch Loss: 1.2276256084442139
Epoch 8, Loss: 4.518041253089905, Final Batch Loss: 1.2628705501556396
Epoch 9, Loss: 4.263685464859009, Final Batch Loss: 1.0135387182235718
Epoch 10, Loss: 4.252756237983704, Final Batch Loss: 1.0044337511062622
Epoch 11, Loss: 4.2537665367126465, Final Batch Loss: 1.01286780834198
Epoch 12, Loss: 4.233704447746277, Final Batch Loss: 1.0008234977722168
Epoch 13, Loss: 4.487145781517029, Final Batch Loss: 1.2670018672943115
Epoch 14, Loss: 4.177056431770325, Final Batch Loss: 0.9624365568161011
Epo

Epoch 122, Loss: 0.34008054714649916, Final Batch Loss: 0.011211256496608257
Epoch 123, Loss: 0.47290199995040894, Final Batch Loss: 0.12013901025056839
Epoch 124, Loss: 0.354988022314501, Final Batch Loss: 0.00018010901112575084
Epoch 125, Loss: 0.28894636617042124, Final Batch Loss: 0.0011332763824611902
Epoch 126, Loss: 0.5158228352665901, Final Batch Loss: 0.13879461586475372
Epoch 127, Loss: 0.3154701800085604, Final Batch Loss: 0.00412446865811944
Epoch 128, Loss: 0.32403409451762855, Final Batch Loss: 2.4199192921514623e-05
Epoch 129, Loss: 0.3140542294131592, Final Batch Loss: 0.0003632839070633054
Epoch 130, Loss: 0.3250706180697307, Final Batch Loss: 0.0008985534077510238
Epoch 131, Loss: 0.37655945055303164, Final Batch Loss: 3.659658250398934e-05
Epoch 132, Loss: 0.3299112804234028, Final Batch Loss: 0.0015070997178554535
Epoch 133, Loss: 0.3276618532836437, Final Batch Loss: 0.061815109103918076
Epoch 134, Loss: 0.3473965502344072, Final Batch Loss: 0.002554608043283224
Ep

Epoch 232, Loss: 0.2228777555283159, Final Batch Loss: 0.001536618685349822
Epoch 233, Loss: 0.27114530478138477, Final Batch Loss: 0.0010725465836003423
Epoch 234, Loss: 0.23272936744615436, Final Batch Loss: 0.0020638848654925823
Epoch 235, Loss: 0.22659371771442238, Final Batch Loss: 0.00012385078298393637
Epoch 236, Loss: 0.26498235389590263, Final Batch Loss: 0.03692613169550896
Epoch 237, Loss: 0.23677814668917563, Final Batch Loss: 4.076874756719917e-05
Epoch 238, Loss: 0.39121851697564125, Final Batch Loss: 0.1548784077167511
Epoch 239, Loss: 0.242490667828406, Final Batch Loss: 6.556489552167477e-06
Epoch 240, Loss: 6.645429119467735, Final Batch Loss: 6.4308624267578125
Epoch 241, Loss: 0.24962211237289011, Final Batch Loss: 0.001426989445462823
Epoch 242, Loss: 0.2960498091124464, Final Batch Loss: 6.05564855504781e-05
Epoch 243, Loss: 0.4010977072903188, Final Batch Loss: 0.00012337400403339416
Epoch 244, Loss: 0.4137201368030219, Final Batch Loss: 5.602679812000133e-05
Epo

Epoch 346, Loss: 0.2542355256155133, Final Batch Loss: 0.014263582415878773
Epoch 347, Loss: 0.29134029150009155, Final Batch Loss: 0.02758176624774933
Epoch 348, Loss: 0.25830355379730463, Final Batch Loss: 0.007691056467592716
Epoch 349, Loss: 0.3097275085747242, Final Batch Loss: 0.07992686331272125
Epoch 350, Loss: 0.24874367413576692, Final Batch Loss: 0.0012869894271716475
Epoch 351, Loss: 0.20964172668755054, Final Batch Loss: 0.015586698427796364
Epoch 352, Loss: 0.20473307464271784, Final Batch Loss: 0.000940476544201374
Epoch 353, Loss: 0.2241666023619473, Final Batch Loss: 0.005592652130872011
Epoch 354, Loss: 0.1932224774354836, Final Batch Loss: 3.45700973412022e-05
Epoch 355, Loss: 0.24622373457532376, Final Batch Loss: 0.0013328249333426356
Epoch 356, Loss: 0.2099092788857888, Final Batch Loss: 5.245195097813848e-06
Epoch 357, Loss: 0.2032376837960328, Final Batch Loss: 7.080780778778717e-05
Epoch 358, Loss: 0.21132015599869192, Final Batch Loss: 0.001934444298967719
Epo

Epoch 454, Loss: 0.21715691311646879, Final Batch Loss: 5.483612312673358e-06
Epoch 455, Loss: 0.17294065561145544, Final Batch Loss: 0.010337735526263714
Epoch 456, Loss: 0.22945553064346313, Final Batch Loss: 0.04769040644168854
Epoch 457, Loss: 0.19028684866862022, Final Batch Loss: 7.152531907195225e-06
Epoch 458, Loss: 0.16223356127738953, Final Batch Loss: 0.0
Epoch 459, Loss: 0.2045821025967598, Final Batch Loss: 0.04851340129971504
Epoch 460, Loss: 0.16371751809856505, Final Batch Loss: 5.280832192511298e-05
Epoch 461, Loss: 0.1731158960610628, Final Batch Loss: 0.011093368753790855
Epoch 462, Loss: 0.15304374363040552, Final Batch Loss: 0.0006451908848248422
Epoch 463, Loss: 0.1656690089730546, Final Batch Loss: 0.00114827963989228
Epoch 464, Loss: 0.19482092617545277, Final Batch Loss: 0.0017254954436793923
Epoch 465, Loss: 0.17942953096644487, Final Batch Loss: 0.00012313561455812305
Epoch 466, Loss: 0.18462165689561516, Final Batch Loss: 0.001865434111095965
Epoch 467, Loss

...

Epoch 4391, Loss: 0.011083220335422084, Final Batch Loss: 0.0
Epoch 4392, Loss: 0.001102122121665161, Final Batch Loss: 0.0
Epoch 4393, Loss: 0.0019239691464463249, Final Batch Loss: 0.0
Epoch 4394, Loss: 0.0022137706982903183, Final Batch Loss: 0.0
Epoch 4395, Loss: 0.0013571754352526, Final Batch Loss: 1.1920928244535389e-07
Epoch 4396, Loss: 0.0022606249840464443, Final Batch Loss: 0.0
Epoch 4397, Loss: 0.0010883200409352867, Final Batch Loss: 2.0265558760002023e-06
Epoch 4398, Loss: 0.008266296455985866, Final Batch Loss: 0.001903866184875369
Epoch 4399, Loss: 0.0008005550116578775, Final Batch Loss: 1.1920928244535389e-07
Epoch 4400, Loss: 0.0010189636113864253, Final Batch Loss: 0.0
Epoch 4401, Loss: 0.011306953354505822, Final Batch Loss: 0.0
Epoch 4402, Loss: 0.00024496040350641124, Final Batch Loss: 0.0
Epoch 4403, Loss: 0.0006115712021710351, Final Batch Loss: 0.0
Epoch 4404, Loss: 0.0037169392744544894, Final Batch Loss: 0.0
Epoch 4405, Loss: 0.0016432074480690062, Final Bat

Epoch 4521, Loss: 0.002966661430008344, Final Batch Loss: 2.3841855067985307e-07
Epoch 4522, Loss: 0.012900881449240842, Final Batch Loss: 0.0
Epoch 4523, Loss: 0.04407589584297966, Final Batch Loss: 0.02378642000257969
Epoch 4524, Loss: 0.0051391213621627685, Final Batch Loss: 1.7881377516459906e-06
Epoch 4525, Loss: 0.00044538391375681385, Final Batch Loss: 0.0
Epoch 4526, Loss: 0.0024790119241444586, Final Batch Loss: 1.1920928244535389e-07
Epoch 4527, Loss: 0.0005230806473193184, Final Batch Loss: 1.1920928244535389e-07
Epoch 4528, Loss: 0.0011390264116926119, Final Batch Loss: 0.0
Epoch 4529, Loss: 0.0014438374346354976, Final Batch Loss: 0.0
Epoch 4530, Loss: 0.0026718045919551514, Final Batch Loss: 0.0
Epoch 4531, Loss: 0.0009750939652803936, Final Batch Loss: 0.0
Epoch 4532, Loss: 0.00029063966758258175, Final Batch Loss: 0.0
Epoch 4533, Loss: 0.0016207859407586511, Final Batch Loss: 0.0
Epoch 4534, Loss: 0.0011189623292011674, Final Batch Loss: 0.0
Epoch 4535, Loss: 0.00035874

Epoch 4646, Loss: 0.00467312684531862, Final Batch Loss: 2.50339189733495e-06
Epoch 4647, Loss: 0.00022174032528710086, Final Batch Loss: 0.0
Epoch 4648, Loss: 0.0004935438319080276, Final Batch Loss: 0.0
Epoch 4649, Loss: 0.00110530924575869, Final Batch Loss: 0.0
Epoch 4650, Loss: 0.00020890381529170554, Final Batch Loss: 0.0
Epoch 4651, Loss: 0.0008146919608407188, Final Batch Loss: 0.0
Epoch 4652, Loss: 0.007569413851797435, Final Batch Loss: 4.172316494077677e-06
Epoch 4653, Loss: 0.0009668748571129981, Final Batch Loss: 0.0
Epoch 4654, Loss: 0.040337168552923686, Final Batch Loss: 9.536738616588991e-07
Epoch 4655, Loss: 0.011347623800247675, Final Batch Loss: 0.0
Epoch 4656, Loss: 0.0005846582569120073, Final Batch Loss: 7.152555099310121e-07
Epoch 4657, Loss: 0.028813331678975374, Final Batch Loss: 0.0
Epoch 4658, Loss: 0.0006091361319704447, Final Batch Loss: 0.0
Epoch 4659, Loss: 0.0011097287488155416, Final Batch Loss: 1.311301275563892e-06
Epoch 4660, Loss: 0.002672705792065

Epoch 4772, Loss: 0.004933653564876295, Final Batch Loss: 3.313963316031732e-05
Epoch 4773, Loss: 0.002027220429219767, Final Batch Loss: 1.1920928244535389e-07
Epoch 4774, Loss: 0.02712808556680102, Final Batch Loss: 0.0
Epoch 4775, Loss: 0.015458411537110806, Final Batch Loss: 0.0
Epoch 4776, Loss: 0.0019495626329444349, Final Batch Loss: 0.0
Epoch 4777, Loss: 0.0012297615357965697, Final Batch Loss: 0.0
Epoch 4778, Loss: 0.017673556176305283, Final Batch Loss: 0.0
Epoch 4779, Loss: 0.014589401987905148, Final Batch Loss: 0.0
Epoch 4780, Loss: 0.0005838256711285794, Final Batch Loss: 1.6569954823353328e-05
Epoch 4781, Loss: 0.0047508478310192, Final Batch Loss: 0.0
Epoch 4782, Loss: 0.000755012544686906, Final Batch Loss: 0.0
Epoch 4783, Loss: 0.0010859325157070998, Final Batch Loss: 0.0
Epoch 4784, Loss: 0.0026287969667464495, Final Batch Loss: 0.0
Epoch 4785, Loss: 0.015639138728147373, Final Batch Loss: 0.0
Epoch 4786, Loss: 0.0009770094256964512, Final Batch Loss: 0.0
Epoch 4787,

Epoch 4896, Loss: 0.0004647314563044347, Final Batch Loss: 0.0
Epoch 4897, Loss: 0.013401321492892748, Final Batch Loss: 0.0
Epoch 4898, Loss: 0.01546031278121518, Final Batch Loss: 0.0
Epoch 4899, Loss: 0.0009261374216293916, Final Batch Loss: 0.0
Epoch 4900, Loss: 0.019774473355937516, Final Batch Loss: 0.0
Epoch 4901, Loss: 0.0004501014045672491, Final Batch Loss: 0.0
Epoch 4902, Loss: 0.004121653033962502, Final Batch Loss: 0.0
Epoch 4903, Loss: 0.003971497681050096, Final Batch Loss: 0.0
Epoch 4904, Loss: 0.013304282780154608, Final Batch Loss: 0.0
Epoch 4905, Loss: 0.025950864364858717, Final Batch Loss: 0.0
Epoch 4906, Loss: 0.00024973707513709087, Final Batch Loss: 0.0
Epoch 4907, Loss: 0.0008577953522035386, Final Batch Loss: 0.0
Epoch 4908, Loss: 0.00047120937961153686, Final Batch Loss: 0.0
Epoch 4909, Loss: 0.008896992425434291, Final Batch Loss: 0.0
Epoch 4910, Loss: 0.009535302735457662, Final Batch Loss: 0.0
Epoch 4911, Loss: 0.0002581466324045323, Final Batch Loss: 0.0


In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[70  0  0]
 [ 1 60  0]
 [ 0  0 57]]
              precision    recall  f1-score   support

           0    0.98592   1.00000   0.99291        70
           1    1.00000   0.98361   0.99174        61
           2    1.00000   1.00000   1.00000        57

    accuracy                        0.99468       188
   macro avg    0.99531   0.99454   0.99488       188
weighted avg    0.99476   0.99468   0.99468       188



In [11]:
n_samples = 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples * 6)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_10 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_11 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_12 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples * 6)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_13 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_14 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_15 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_16 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_17 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_18 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples * 6) + 1

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9, fake_features_10, fake_features_11, fake_features_12,
                               fake_features_13, fake_features_14, fake_features_15, fake_features_16, fake_features_17, fake_features_18))
fake_labels = np.concatenate((y_1, y_2, y_3))

fake_features = torch.Tensor(fake_features)

In [12]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[59  0  1]
 [ 0 60  0]
 [ 0  0 60]]
              precision    recall  f1-score   support

         0.0    1.00000   0.98333   0.99160        60
         1.0    1.00000   1.00000   1.00000        60
         2.0    0.98361   1.00000   0.99174        60

    accuracy                        0.99444       180
   macro avg    0.99454   0.99444   0.99444       180
weighted avg    0.99454   0.99444   0.99444       180



# User Classifier

In [13]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [14]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    elif y[k] == 7:
        y[k] = 3
    elif y[k] == 8:
        y[k] = 4
    else:
        y[k] = 5

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [16]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 7.324767351150513, Final Batch Loss: 1.9476697444915771
Epoch 2, Loss: 7.087306380271912, Final Batch Loss: 1.713757038116455
Epoch 3, Loss: 7.0897133350372314, Final Batch Loss: 1.7141481637954712
Epoch 4, Loss: 7.303276181221008, Final Batch Loss: 1.9305531978607178
Epoch 5, Loss: 7.312042713165283, Final Batch Loss: 1.94318425655365
Epoch 6, Loss: 7.326673269271851, Final Batch Loss: 1.9543445110321045
Epoch 7, Loss: 7.180524110794067, Final Batch Loss: 1.8120821714401245
Epoch 8, Loss: 6.924331784248352, Final Batch Loss: 1.5617165565490723
Epoch 9, Loss: 7.269026041030884, Final Batch Loss: 1.9064658880233765
Epoch 10, Loss: 6.917933225631714, Final Batch Loss: 1.5517066717147827
Epoch 11, Loss: 7.081150650978088, Final Batch Loss: 1.7208610773086548
Epoch 12, Loss: 7.050141453742981, Final Batch Loss: 1.6901044845581055
Epoch 13, Loss: 7.259033679962158, Final Batch Loss: 1.8995311260223389
Epoch 14, Loss: 7.040647983551025, Final Batch Loss: 1.6815738677978516
Epo

Epoch 120, Loss: 4.9789615869522095, Final Batch Loss: 0.8175690174102783
Epoch 121, Loss: 5.128903746604919, Final Batch Loss: 1.0258671045303345
Epoch 122, Loss: 5.493600368499756, Final Batch Loss: 1.2993334531784058
Epoch 123, Loss: 7.355509877204895, Final Batch Loss: 3.118911027908325
Epoch 124, Loss: 6.32703173160553, Final Batch Loss: 2.1168880462646484
Epoch 125, Loss: 5.717969536781311, Final Batch Loss: 1.6001087427139282
Epoch 126, Loss: 5.9138407707214355, Final Batch Loss: 1.7884864807128906
Epoch 127, Loss: 5.617858052253723, Final Batch Loss: 1.486894965171814
Epoch 128, Loss: 5.483014345169067, Final Batch Loss: 1.33988356590271
Epoch 129, Loss: 6.0257333517074585, Final Batch Loss: 1.9047983884811401
Epoch 130, Loss: 4.613044112920761, Final Batch Loss: 0.47653111815452576
Epoch 131, Loss: 5.55274510383606, Final Batch Loss: 1.4567670822143555
Epoch 132, Loss: 5.189761400222778, Final Batch Loss: 1.1275300979614258
Epoch 133, Loss: 4.804865479469299, Final Batch Loss:

Epoch 242, Loss: 3.999774068593979, Final Batch Loss: 0.4275122582912445
Epoch 243, Loss: 5.22041392326355, Final Batch Loss: 1.651489496231079
Epoch 244, Loss: 3.7167284190654755, Final Batch Loss: 0.22315213084220886
Epoch 245, Loss: 4.587936878204346, Final Batch Loss: 1.1303898096084595
Epoch 246, Loss: 5.303292274475098, Final Batch Loss: 1.7779109477996826
Epoch 247, Loss: 5.470344662666321, Final Batch Loss: 1.9378678798675537
Epoch 248, Loss: 5.16180944442749, Final Batch Loss: 1.6224594116210938
Epoch 249, Loss: 5.15681529045105, Final Batch Loss: 1.7483646869659424
Epoch 250, Loss: 4.952137112617493, Final Batch Loss: 1.5259270668029785
Epoch 251, Loss: 4.8489216566085815, Final Batch Loss: 1.4981955289840698
Epoch 252, Loss: 5.152587056159973, Final Batch Loss: 1.716200828552246
Epoch 253, Loss: 4.379698693752289, Final Batch Loss: 0.8585501313209534
Epoch 254, Loss: 5.444775223731995, Final Batch Loss: 1.9153568744659424
Epoch 255, Loss: 4.91340959072113, Final Batch Loss: 

Epoch 364, Loss: 3.9531408548355103, Final Batch Loss: 1.098373532295227
Epoch 365, Loss: 3.4954793453216553, Final Batch Loss: 0.5587245225906372
Epoch 366, Loss: 4.165323615074158, Final Batch Loss: 1.2437822818756104
Epoch 367, Loss: 3.855770766735077, Final Batch Loss: 0.9599010944366455
Epoch 368, Loss: 3.5415871143341064, Final Batch Loss: 0.6778891682624817
Epoch 369, Loss: 4.723100185394287, Final Batch Loss: 1.8740044832229614
Epoch 370, Loss: 3.878543257713318, Final Batch Loss: 1.0803393125534058
Epoch 371, Loss: 3.195245146751404, Final Batch Loss: 0.28364914655685425
Epoch 372, Loss: 3.7890482544898987, Final Batch Loss: 0.9882201552391052
Epoch 373, Loss: 4.673769116401672, Final Batch Loss: 1.7210179567337036
Epoch 374, Loss: 3.114308178424835, Final Batch Loss: 0.3466287851333618
Epoch 375, Loss: 4.445478856563568, Final Batch Loss: 1.5914852619171143
Epoch 376, Loss: 4.530440151691437, Final Batch Loss: 1.7089293003082275
Epoch 377, Loss: 5.457998752593994, Final Batch

Epoch 485, Loss: 3.034577190876007, Final Batch Loss: 0.299923837184906
Epoch 486, Loss: 2.7171144410967827, Final Batch Loss: 0.06235813349485397
Epoch 487, Loss: 2.7776255756616592, Final Batch Loss: 0.08908297121524811
Epoch 488, Loss: 2.951182872056961, Final Batch Loss: 0.33158883452415466
Epoch 489, Loss: 4.5480599999427795, Final Batch Loss: 1.807560682296753
Epoch 490, Loss: 4.118229150772095, Final Batch Loss: 1.403193473815918
Epoch 491, Loss: 4.6420938372612, Final Batch Loss: 2.005972146987915
Epoch 492, Loss: 4.128359019756317, Final Batch Loss: 1.5051510334014893
Epoch 493, Loss: 7.209329724311829, Final Batch Loss: 4.594730854034424
Epoch 494, Loss: 4.290566086769104, Final Batch Loss: 1.4914677143096924
Epoch 495, Loss: 3.2669522166252136, Final Batch Loss: 0.567240834236145
Epoch 496, Loss: 4.043312191963196, Final Batch Loss: 1.305344820022583
Epoch 497, Loss: 4.519927680492401, Final Batch Loss: 1.7692625522613525
Epoch 498, Loss: 4.562558174133301, Final Batch Loss:

...

Epoch 4499, Loss: 1.4956259345635772, Final Batch Loss: 0.012197545729577541
Epoch 4500, Loss: 1.3970561623573303, Final Batch Loss: 0.0
Epoch 4501, Loss: 1.4145292527973652, Final Batch Loss: 0.03935890272259712
Epoch 4502, Loss: 2.2314994037151337, Final Batch Loss: 0.9034202694892883
Epoch 4503, Loss: 1.4015132738422835, Final Batch Loss: 9.63164638960734e-05
Epoch 4504, Loss: 1.9269347190856934, Final Batch Loss: 0.5023369193077087
Epoch 4505, Loss: 2.493835359811783, Final Batch Loss: 0.9892603158950806
Epoch 4506, Loss: 2.7837541699409485, Final Batch Loss: 1.4230319261550903
Epoch 4507, Loss: 1.479354154318571, Final Batch Loss: 0.022569458931684494
Epoch 4508, Loss: 1.569135569036007, Final Batch Loss: 0.07348185032606125
Epoch 4509, Loss: 4.072472870349884, Final Batch Loss: 2.444058418273926
Epoch 4510, Loss: 1.6980252638459206, Final Batch Loss: 0.07445643097162247
Epoch 4511, Loss: 3.233040750026703, Final Batch Loss: 1.649332046508789
Epoch 4512, Loss: 2.9787800312042236, 

Epoch 4612, Loss: 1.3255667071789503, Final Batch Loss: 0.011426953598856926
Epoch 4613, Loss: 1.3093066215514568, Final Batch Loss: 3.576278118089249e-07
Epoch 4614, Loss: 1.3621832567732781, Final Batch Loss: 0.0012809175532311201
Epoch 4615, Loss: 2.515632599592209, Final Batch Loss: 1.2441163063049316
Epoch 4616, Loss: 1.2646222419571131, Final Batch Loss: 0.00024125049822032452
Epoch 4617, Loss: 2.9124240279197693, Final Batch Loss: 1.5435022115707397
Epoch 4618, Loss: 1.3853197139687836, Final Batch Loss: 0.001167092937976122
Epoch 4619, Loss: 1.5934962865503621, Final Batch Loss: 0.00011205045302631333
Epoch 4620, Loss: 1.850068163126707, Final Batch Loss: 0.02047206088900566
Epoch 4621, Loss: 2.17423614859581, Final Batch Loss: 0.3171752393245697
Epoch 4622, Loss: 1.712124544661492, Final Batch Loss: 0.0023834886960685253
Epoch 4623, Loss: 1.416985738556832, Final Batch Loss: 0.00271071819588542
Epoch 4624, Loss: 1.424583401530981, Final Batch Loss: 0.01356055960059166
Epoch 46

Epoch 4722, Loss: 1.5628516469150782, Final Batch Loss: 0.026273855939507484
Epoch 4723, Loss: 1.5986979380249977, Final Batch Loss: 0.08789024502038956
Epoch 4724, Loss: 1.4937454746104777, Final Batch Loss: 0.0007737264968454838
Epoch 4725, Loss: 1.4394169971346855, Final Batch Loss: 0.007965229451656342
Epoch 4726, Loss: 1.559566006064415, Final Batch Loss: 0.07059817016124725
Epoch 4727, Loss: 2.8800743520259857, Final Batch Loss: 1.359267234802246
Epoch 4728, Loss: 1.5443990221247077, Final Batch Loss: 0.004915415309369564
Epoch 4729, Loss: 3.0639306902885437, Final Batch Loss: 1.5543444156646729
Epoch 4730, Loss: 1.5692817606031895, Final Batch Loss: 0.030841592699289322
Epoch 4731, Loss: 1.4965881591197103, Final Batch Loss: 0.0018807833548635244
Epoch 4732, Loss: 1.9793227910995483, Final Batch Loss: 0.508560061454773
Epoch 4733, Loss: 1.5254370141774416, Final Batch Loss: 0.011141704395413399
Epoch 4734, Loss: 1.545297022908926, Final Batch Loss: 0.020097550004720688
Epoch 473

Epoch 4832, Loss: 1.4903898340417072, Final Batch Loss: 0.0005270045949146152
Epoch 4833, Loss: 1.461052279919386, Final Batch Loss: 0.04411197826266289
Epoch 4834, Loss: 2.4161621630191803, Final Batch Loss: 1.0057334899902344
Epoch 4835, Loss: 2.628078430891037, Final Batch Loss: 1.238690733909607
Epoch 4836, Loss: 2.4497110545635223, Final Batch Loss: 1.0872387886047363
Epoch 4837, Loss: 2.266256332397461, Final Batch Loss: 0.9472771286964417
Epoch 4838, Loss: 1.4398313760757446, Final Batch Loss: 0.0
Epoch 4839, Loss: 3.2321803867816925, Final Batch Loss: 1.8004493713378906
Epoch 4840, Loss: 1.9194468557834625, Final Batch Loss: 0.5288419127464294
Epoch 4841, Loss: 1.4355807833489962, Final Batch Loss: 0.0006090931710787117
Epoch 4842, Loss: 2.330797702074051, Final Batch Loss: 0.9796951413154602
Epoch 4843, Loss: 3.9526856541633606, Final Batch Loss: 2.5467028617858887
Epoch 4844, Loss: 1.5983734279870987, Final Batch Loss: 0.13526864349842072
Epoch 4845, Loss: 1.3780344645492733,

Epoch 4947, Loss: 1.36879447940737, Final Batch Loss: 0.011174359358847141
Epoch 4948, Loss: 1.3057560920715332, Final Batch Loss: 0.016651928424835205
Epoch 4949, Loss: 1.4001265991755645, Final Batch Loss: 0.00011872540198964998
Epoch 4950, Loss: 2.891960710287094, Final Batch Loss: 1.6043767929077148
Epoch 4951, Loss: 2.0106993913650513, Final Batch Loss: 0.7250180244445801
Epoch 4952, Loss: 1.5788757801055908, Final Batch Loss: 0.2701202630996704
Epoch 4953, Loss: 1.4421348869800568, Final Batch Loss: 0.13398820161819458
Epoch 4954, Loss: 1.3194102048873901, Final Batch Loss: 0.007787108421325684
Epoch 4955, Loss: 2.144783675670624, Final Batch Loss: 0.8424031138420105
Epoch 4956, Loss: 1.353608574718237, Final Batch Loss: 0.04604613408446312
Epoch 4957, Loss: 1.3056495431810617, Final Batch Loss: 0.009898269549012184
Epoch 4958, Loss: 2.1386231780052185, Final Batch Loss: 0.6993080973625183
Epoch 4959, Loss: 3.119908422231674, Final Batch Loss: 1.7431716918945312
Epoch 4960, Loss:

In [17]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[32  0  0  0  0  0]
 [ 1 21  1  3  1  1]
 [ 0  0 25  2  0  1]
 [ 0  0  2 33  0  0]
 [14  0  4  1 13  0]
 [ 0  0  0  0  1 32]]
              precision    recall  f1-score   support

           0    0.68085   1.00000   0.81013        32
           1    1.00000   0.75000   0.85714        28
           2    0.78125   0.89286   0.83333        28
           3    0.84615   0.94286   0.89189        35
           4    0.86667   0.40625   0.55319        32
           5    0.94118   0.96970   0.95522        33

    accuracy                        0.82979       188
   macro avg    0.85268   0.82694   0.81682       188
weighted avg    0.85143   0.82979   0.81754       188



In [18]:
fake_labels = [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [5] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [5] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [5] * n_samples
fake_labels = np.asarray(fake_labels)

In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(fake_labels, preds.cpu()))
print(metrics.classification_report(fake_labels, preds.cpu(), digits = 5))

[[23  0  1  2  4  0]
 [ 3 17  0  3  5  2]
 [ 0  0 17  7  2  4]
 [ 0  2  1 24  3  0]
 [ 2  0  0  0 24  4]
 [ 1  0  3  0  4 22]]
              precision    recall  f1-score   support

           0    0.79310   0.76667   0.77966        30
           1    0.89474   0.56667   0.69388        30
           2    0.77273   0.56667   0.65385        30
           3    0.66667   0.80000   0.72727        30
           4    0.57143   0.80000   0.66667        30
           5    0.68750   0.73333   0.70968        30

    accuracy                        0.70556       180
   macro avg    0.73103   0.70556   0.70517       180
weighted avg    0.73103   0.70556   0.70517       180

